# Trabajo Final de Master

Este notebook contiene los códigos y desarrollos para estudiar diferentes modelos de movilidad y aplocarlos para el caso puntual de la ciudad de Bogotá

### Primera Parte. Tratamiento de datos de la encuesta de movilidad del 2019

En esta primera parte se hace un primer llamado a la encuesta de movilidad del año 2019 de Bogotá, en donde se estudiará la calidad de la fuente de datos y se crearan matrices origen destino para todos los medios de transporte, y así verificar si los datos se ajustan a un modelo gravitacional.

In [ ]:
# Calling the libreries 
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geopy.distance

# Declaring the paths of the trips data
trips_path = r'./Encuesta de Movilidad 2019/EODH/Archivos_CSV/ViajesEODH2019.csv'
# Path of the population data
houses_path = r'./Encuesta de Movilidad 2019/EODH/Archivos_CSV/HogaresEODH2019.csv'
# Path of the ZATs shapefiles
shape_path = r'./Encuesta de Movilidad 2019/Zonificacion_(shapefiles)/ZONAS/ZONAS/ZAT.shp'
# Path of the streets shapefile
streets_path = r'./Encuesta de Movilidad 2019/Zonificacion_(shapefiles)/Malla_Vial_Integral_Bogota_D_C/Malla_Vial_Integral_Bogota_D_C.shp'
# Path of the "Localidades" shapefile
loc_path = r'./Encuesta de Movilidad 2019/Zonificacion_(shapefiles)/loca/Loca.shp'
# Converting the CSV in a pandas dataframe
trips_df = pd.read_csv(trips_path, sep = ';')
# Converting the population data to dataframe
houses_df = pd.read_csv(houses_path, sep= ';', low_memory=False)
# Calling the ZAT's shapefile
zats_map = gpd.read_file(shape_path)
# Calling the streets shapefile
#streets_map = gpd.read_file(streets_path)
# Calling the "Localidades" shapefile
loc_map = gpd.read_file(loc_path)

# Function which plots the map of Bogotá 
def map(zat):
    fig, ax = plt.subplots(1, 1, figsize=(25, 25))
    loc_map.boundary.plot(ax=ax, color='black')
    zats_map[zats_map['ZAT']==zat].plot(color='g', ax=ax)
    plt.plot()

# Computing the centroid of each zat
zats_map['center'] = zats_map.to_crs('+proj=cea').centroid.to_crs(zats_map.crs)

Ok como una primera aproximación no se van a filtrar los datos por tiempos de viaje, ni velocidades, por lo cual, en esta primera parte se tendrán en cuenta todos los datos.
La otra cuestión es que, como me sucedio en mi TFG, el muestreo sobre los de mas municipios está mal hecho, con los datos utilizados Bojaca tendría una población mayor a Faca y eso obviamente no es congruente con la vida real y el modelo estaría incorrecto, por lo cual, de nuevo, se van a utilizar solo datos en donde los vijes sean en Bogotá o Soacha.

In [ ]:
# Filtering the data in order to take only the trips in Bogotá and Soacha
trips_df = trips_df[(trips_df['mun_origen'] == 11001) | (trips_df['mun_origen'] == 25754)]
trips_df = trips_df[(trips_df['mun_destino'] == 11001) | (trips_df['mun_destino'] == 25754)]
houses_df = houses_df[(houses_df['municipio'] == 11001) | (houses_df['municipio'] == 25754)]

In [ ]:
# Computing the population per zat
population = houses_df[['zat_hogar', 'p7_total_personas']]
population = population.groupby('zat_hogar').agg('sum').sort_values(by='p7_total_personas', ascending=False)

Dado que el modelo gravitacional escala como $r^{-1}$, no podemos tener en cuenta viajes dentro de una misma ZAT, por lo cual estos serán filtrados.

In [ ]:
# Computing the number of trips between diferent zats 
num_trips_df = trips_df[['zat_origen', 'zat_destino']]
#num_trips_df['trips'] = np.ones(len(num_trips_df)) 
num_trips_df.loc[0:, ['trips']] = 1
num_trips_df = (num_trips_df.groupby(['zat_origen', 'zat_destino'], as_index=False)['trips']
                            .agg('sum')
                            .sort_values(by='trips', ascending=False))
num_trips_df = num_trips_df[num_trips_df['zat_destino'] != num_trips_df['zat_origen']]

Dado que el modelo gravitacional es simetrico, debemos sumar los viajes de "ida y de regreso" entre dos ZATs, por lo cual, vamos a convertir el numero de viajes en una matriz y la sumamos con su transpuesta para simetrizar esta matriz, y luego la convertimos en columna de nuevo. 

In [ ]:
# Symmetrizing the number of trips between each zat
table = pd.pivot_table(num_trips_df, values='trips', index='zat_origen', columns='zat_destino').fillna(0)
table = table + table.T

sim_num_trips = (table.stack()
                      .to_frame()
                      .reset_index()
                      .rename(columns={'level_0': 'zat_origen', 'level_1': 'zat_destino', 0: 'trips'})
                      .sort_values(by='trips', ascending=False))
sim_num_trips = sim_num_trips[sim_num_trips['trips']>0]
sim_num_trips

,zat_origen,zat_destino,trips
466083,530.0,551.0,139.0
484899,551.0,530.0,139.0
672601,776.0,775.0,112.0
671705,775.0,776.0,112.0
621606,714.0,1050.0,105.0
...,...,...,...
399598,454.0,442.0,1.0
468555,533.0,329.0,1.0
239590,274.0,95.0,1.0
387208,440.0,614.0,1.0


In [ ]:
# Adding the centroid coordinates to the df

num_dis_trips = (num_trips_df.merge(zats_map[['ZAT', 'center']], left_on='zat_origen', right_on='ZAT')
                             .rename(columns={'center': 'Origin_center'})
                             .drop(columns='ZAT'))
num_dis_trips = (num_dis_trips.merge(zats_map[['ZAT', 'center']], left_on='zat_destino', right_on='ZAT')
                             .rename(columns={'center': 'Destination_center'})
                             .drop(columns='ZAT'))
num_dis_trips.sort_values(by='trips', ascending=False)

,zat_origen,zat_destino,trips,Origin_center,Destination_center
0,530.0,551.0,73,POINT (-74.17593 4.63213),POINT (-74.18166 4.62800)
21960,551.0,530.0,66,POINT (-74.18166 4.62800),POINT (-74.17593 4.63213)
21838,775.0,776.0,56,POINT (-74.19736 4.58356),POINT (-74.19802 4.57757)
27150,776.0,775.0,56,POINT (-74.19802 4.57757),POINT (-74.19736 4.58356)
33242,714.0,1050.0,53,POINT (-74.16040 4.55118),POINT (-74.15788 4.52832)
...,...,...,...,...,...
22992,701.0,620.0,1,POINT (-74.09146 4.51337),POINT (-74.15452 4.58421)
22993,413.0,620.0,1,POINT (-74.16893 4.64880),POINT (-74.15452 4.58421)
22995,661.0,620.0,1,POINT (-74.14492 4.56405),POINT (-74.15452 4.58421)
22996,694.0,620.0,1,POINT (-74.08985 4.52125),POINT (-74.15452 4.58421)


In [ ]:
# Adding the centroid coordinates to the symmetrical df

num_dis_sim = (sim_num_trips.merge(zats_map[['ZAT', 'center']], left_on='zat_origen', right_on='ZAT')
                             .rename(columns={'center': 'Origin_center'})
                             .drop(columns='ZAT'))
num_dis_sim = (num_dis_sim.merge(zats_map[['ZAT', 'center']], left_on='zat_destino', right_on='ZAT')
                             .rename(columns={'center': 'Destination_center'})
                             .drop(columns='ZAT'))
num_dis_sim.sort_values(by='trips', ascending=False)

,zat_origen,zat_destino,trips,Origin_center,Destination_center
0,530.0,551.0,139.0,POINT (-74.17593 4.63213),POINT (-74.18166 4.62800)
29708,551.0,530.0,139.0,POINT (-74.18166 4.62800),POINT (-74.17593 4.63213)
36271,776.0,775.0,112.0,POINT (-74.19802 4.57757),POINT (-74.19736 4.58356)
25603,775.0,776.0,112.0,POINT (-74.19736 4.58356),POINT (-74.19802 4.57757)
45336,1050.0,714.0,105.0,POINT (-74.15788 4.52832),POINT (-74.16040 4.55118)
...,...,...,...,...,...
20317,433.0,842.0,1.0,POINT (-74.09286 4.60852),POINT (-74.08922 4.67644)
49496,153.0,399.0,1.0,POINT (-74.03147 4.69940),POINT (-74.14835 4.67276)
49495,305.0,399.0,1.0,POINT (-74.13276 4.68631),POINT (-74.14835 4.67276)
49494,722.0,399.0,1.0,POINT (-74.15299 4.53344),POINT (-74.14835 4.67276)


Proxima tarea: Convertir la matriz origne destino ya simetrica en un dataframe tipo columnas para tener la cantidad de viajes totales entre dos ZATs.

In [ ]:
np.ones(4)

array([1., 1., 1., 1.])

In [ ]:
num_trips_df

,zat_origen,zat_destino,trips
39837,530.0,551.0,73.0
41417,551.0,530.0,66.0
52503,776.0,775.0,56.0
52394,775.0,776.0,56.0
49695,714.0,1050.0,53.0
...,...,...,...
23080,316.0,122.0,1.0
23081,316.0,207.0,1.0
23082,316.0,241.0,1.0
23083,316.0,245.0,1.0


In [ ]:
num_trips_df

,zat_origen,zat_destino,trips
39837,530.0,551.0,73.0
41417,551.0,530.0,66.0
52503,776.0,775.0,56.0
52394,775.0,776.0,56.0
49695,714.0,1050.0,53.0
...,...,...,...
23080,316.0,122.0,1.0
23081,316.0,207.0,1.0
23082,316.0,241.0,1.0
23083,316.0,245.0,1.0
